In [1]:
!pip install transformers
from transformers import AutoTokenizer, AutoModelForTokenClassification,AutoModel



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.1 MB/s eta 0:00:00


Intializing Model Tokenizer and import libraries


In [2]:
modelName = "emilyalsentzer/Bio_ClinicalBERT"
tokenizer = AutoTokenizer.from_pretrained(modelName)


In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from tqdm import tqdm
from torch.optim import Adam,AdamW
import torch.nn.functional as F
from collections import Counter
import itertools
from collections import defaultdict
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score,f1_score, precision_score, recall_score
torch.random.manual_seed(6)
np.random.seed(6)

C:\Users\ASUS\anaconda3\envs\shubham\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


Please input paths here to dataset G1, G2, G3 and G4

In [5]:
pathToG1 = 'drive/MyDrive/nerDataset/G1.xlsx'
pathToG2 = 'drive/MyDrive/nerDataset/G2.xlsx'
pathToG3 = 'drive/MyDrive/nerDataset/G3.xlsx'
pathToG4 = 'drive/MyDrive/nerDataset/G1.xlsx'

1) Reading data files as dataframe.
2) LowerCasing
3) Splitting into train, test & validation sets with 80%, 10% and 10% respectively


In [6]:
dfG1 = pd.read_excel(pathToG1, index_col=0)
dfG1.dropna(subset=['text','tags'], inplace=True)
dfG1.reset_index(inplace = True)
#try lower casing and uncased model
dfG1['tags'] = dfG1['tags'].apply(lambda x: x.lower())
dfG1['text'] = dfG1['text'].apply(lambda x: x.lower())
dfTrainG1,dfValG1,dfTestG1 = np.split(dfG1.sample(frac=1, random_state=42),[int(.8 * len(dfG1)),int(.9 * len(dfG1))])
len(dfTrainG1),len(dfTestG1),len(dfValG1)

(5884, 736, 736)

In [7]:
dfG2 = pd.read_excel(pathToG2, index_col=0)
dfG2.dropna(subset=['text','tags'], inplace=True)
dfG2.reset_index(inplace = True)
#try lower casing and uncased model
dfG2['tags'] = dfG2['tags'].apply(lambda x: x.lower())
dfG2['text'] = dfG2['text'].apply(lambda x: x.lower())
dfTrainG2,dfValG2,dfTestG2 = np.split(dfG2.sample(frac=1, random_state=42),[int(.8 * len(dfG2)),int(.9 * len(dfG2))])
len(dfTrainG2),len(dfTestG2),len(dfValG2)

(5163, 646, 645)

In [8]:
dfG3 = pd.read_excel(pathToG3, index_col=0)
dfG3.dropna(subset=['text','tags'], inplace=True)
dfG3.reset_index(inplace = True)
#try lower casing and uncased model
dfG3['tags'] = dfG3['tags'].apply(lambda x: x.lower())
dfG3['text'] = dfG3['text'].apply(lambda x: x.lower())
dfTrainG3,dfValG3,dfTestG3 = np.split(dfG3.sample(frac=1, random_state=42),[int(.8 * len(dfG3)),int(.9 * len(dfG3))])
len(dfTrainG3),len(dfTestG3),len(dfValG3)

(5021, 628, 628)

In [9]:
dfG4 = pd.read_excel(pathToG4, index_col=0)
dfG4.dropna(subset=['text','tags'], inplace=True)
dfG4.reset_index(inplace = True)
#try lower casing and uncased model
dfG4['tags'] = dfG4['tags'].apply(lambda x: x.lower())
dfG4['text'] = dfG4['text'].apply(lambda x: x.lower())
dfTrainG4,dfValG4,dfTestG4 = np.split(dfG4.sample(frac=1, random_state=42),[int(.8 * len(dfG4)),int(.9 * len(dfG4))])
len(dfTrainG4),len(dfTestG4),len(dfValG4)

(5884, 736, 736)

In [10]:
dfTrainG1.head()

,index,ID,tags,text
1893,1893,NCT02945800,1:38:chronic_disease,psychiatric illness/social situations that wou...
4384,4384,NCT02464878,"1:35:chronic_disease,,,",selective or severe iga deficiency (levels < 5...
6820,6820,NCT02059239,"84:92:cancer,104:122:cancer,136:175:treatment",must have histologically or cytologically conf...
5905,5905,NCT02584647,"10:33:chronic_disease,35:61:chronic_disease,,1...",unstable coronary artery disease (myocardial i...
1669,1669,NCT02826577,1:19:chronic_disease,psychotic disorder


In [11]:
dfTestG1.head()

,index,ID,tags,text
2681,2681,NCT02099240,32:45:chronic_disease,pathology report indicative of osteomyelitis
2993,2993,NCT02309580,"29:60:treatment,",patients who have undergone autologous stem ce...
5512,5512,NCT03176979,",23:31:treatment,33:40:treatment,45:62:treatme...","women with history of surgical, medical, or ra..."
6313,6313,NCT02419495,"1:13:treatment,18:31:treatment,45:72:treatment...","chemotherapy, or immunotherapy or any other sy..."
1191,1191,NCT01884740,",,,144:153:treatment",patients of reproductive age must agree to use...


In [12]:
dfValG1.head()

,index,ID,tags,text
1279,1279,NCT02373644,1:9:chronic_disease,diabetes
6565,6565,NCT02266719,"12:38:chronic_disease,45:67:chronic_disease,69...",history of connective tissue disorder (e.g. va...
7205,7205,NCT01464034,"17:35:treatment,37:44:treatment,59:76:treatmen...","exposure to any prior chemotherapy, steroid us..."
1555,1555,NCT01625234,24:46:chronic_disease,clinically significant cardiovascular disease
6651,6651,NCT02188745,"1:16:treatment,,34:44:cancer,48:72:cancer,",surgical biopsy of ≥1 site(s) of metastatic or...


Creating a dictionary of tags to get the label encoding. Also creating a reverse dictionary to get labels from actual tags

In [13]:
uniqueLabels =  ['cancer','chronic_disease','treatment', 'allergy_name', 'other']
labelDict = {i: l for i, l in enumerate(uniqueLabels)}
reverseLabelDict = {v: k for k, v in labelDict.items()}
labelDict

{0: 'cancer',
 1: 'chronic_disease',
 2: 'treatment',
 3: 'allergy_name',
 4: 'other'}

We currently have the tags in index form, for eg: l:r:tag, where l, r are indices in text. Hence we need to process this to a standard format in order to feed this to our neural network model

As a first step in preprocessing, we convert the tags, for each entry in the data, to a list of tuples with 2nd element as tag and first element as the corresponding word(s) in text. eg: [("pneumonia","chronic_disease"),("skin_allergy","allergy_name"),....]

In the next step, we tokenize the text into tokens/subwords using the tokenizer extracted at the start from biobert Model. Tokens are integers, we convert them to string tokens using tokenizer.convert_ids_to_tokens function. Now we iterate over tokens, for each entry in the data, and create words out of them by merging the tokens of a single word. This is done based on the logic that sequential tokens of a continuous word start with "##" when BERT tokenizer is used.

Now we have the list of words and tags for each entry. So we iterate over these and a create a list of word labels which is dictionary of labels corresponding to each word id (index of word in the text). If word has no label in the tags list, then we assign it as "other". Now for each token, we can access the word id, i.e. we know what index word the token belongs to, hence we can create a list of labels for the tokens of text for each entry. Hence, the tokens and the label list of tokens forms our main input

In [14]:
def getLabelIds(wordIds,labelMap):
    labelIds=[]
    for x in wordIds:
        if(x is None):labelIds.append(-100)
        else:labelIds.append(reverseLabelDict.get(labelMap.get(x,'other')))
    return labelIds
def extractTag(tag,text):
    c=tag.find(':')
    l = int(tag[0:c])-1
    c1=tag.find(':',c+1)
    r=int(tag[c+1:c1])-1
    label=tag[c1+1:]
    # if not isinstance(text, str):print(text)
    return text[l:r],label,l,r-1
def preprocess(df):
    df['tags'] = df['tags'].apply(lambda x: x.replace(" ",""))
    tags=[]
    for i,row in df.iterrows():
        # print(row)
        tagRow = []
        text = row['text']
        prev=-1
        for tag in row['tags'].split(','):
            if(len(tag)==0):
                continue
            else:
                a=extractTag(tag,text)
                if(prev<a[2] and prev<a[3]):
                    tagRow.append((a[0],a[1]))
                prev=a[3]
        tags.append(np.array(tagRow))
    text = df["text"].values.tolist()
    tokens= [tokenizer(text, padding = "max_length", truncation = True, return_tensors = "pt",max_length=512) for text in text]
    decoded = [tokenizer.convert_ids_to_tokens(b['input_ids'][0]) for b in tokens]
    wordsInText = []
    for d in decoded:
        words = []
        current_word = ""
        for token in d:
            if token in ['[PAD]','[CLS]','[SEP]']: continue
            if token.startswith("##"):  # Subword continuation
                current_word += token[2:]
            else:  # New word
                if current_word:
                    words.append(current_word)
                current_word = token

        # Add the last word
        if current_word:
            words.append(current_word)
        wordsInText.append(words)
    wordLabels =[]
    for l,m in zip(wordsInText,tags):
    # l,m=wordsInText[0],tags[0]
        wordIndex=0
        d={}
        for x in m:
            x[0]=x[0].replace(" ","")
            index=0
            while(wordIndex<len(l) and index<len(x[0])):
                if(x[0][index:].startswith(l[wordIndex])):
                    # print(l[wordIndex])
                    index+=len(l[wordIndex])
                    d[wordIndex] = x[1]
                    wordIndex+=1
                else:
                    # print(" trye")
                    d[wordIndex]='other'
                    # if(len(wordLabelsTrain)==0):print(wordIndex,l[wordIndex],d)
                    wordIndex+=1
        wordLabels.append(d)
    labels = [getLabelIds(token.word_ids(),labelMap) for token,labelMap in zip(tokens,wordLabels)]
    return tokens,labels,wordsInText,tags,wordLabels

Preprocessing datasets using the methodology described above

In [15]:
tokensTrainG1,labelsTrainG1,wordsInTextTrainG1,tagsTrainG1,wordLabelsTrainG1 = preprocess(dfTrainG1)
tokensTestG1,labelsTestG1,wordsInTextTestG1,tagsTestG1,wordLabelsTestG1 = preprocess(dfTestG1)
tokensValG1,labelsValG1,wordsInTextValG1,tagsValG1,wordLabelsValG1 = preprocess(dfValG1)

In [16]:
tokensTrainG2,labelsTrainG2,wordsInTextTrainG2,tagsTrainG2,wordLabelsTrainG2 = preprocess(dfTrainG2)
tokensTestG2,labelsTestG2,wordsInTextTestG2,tagsTestG2,wordLabelsTestG2 = preprocess(dfTestG2)
tokensValG2,labelsValG2,wordsInTextValG2,tagsValG2,wordLabelsValG2 = preprocess(dfValG2)

In [17]:
tokensTrainG3,labelsTrainG3,wordsInTextTrainG3,tagsTrainG3,wordLabelsTrainG3 = preprocess(dfTrainG3)
tokensTestG3,labelsTestG3,wordsInTextTestG3,tagsTestG3,wordLabelsTestG3 = preprocess(dfTestG3)
tokensValG3,labelsValG3,wordsInTextValG3,tagsValG3,wordLabelsValG3 = preprocess(dfValG3)

In [18]:
tokensTrainG4,labelsTrainG4,wordsInTextTrainG4,tagsTrainG4,wordLabelsTrainG4 = preprocess(dfTrainG4)
tokensTestG4,labelsTestG4,wordsInTextTestG4,tagsTestG4,wordLabelsTestG4 = preprocess(dfTestG4)
tokensValG4,labelsValG4,wordsInTextValG4,tagsValG4,wordLabelsValG4 = preprocess(dfValG4)

In [19]:
len(tagsTrainG3),len(tagsValG3),len(tagsTestG3)

(5021, 628, 628)

In [20]:
# sum([len(x.values())-list(x.values()).count("others") for x in wordLabelsTrainG1])

Finding the count of tags for each dataset

In [21]:
tagCountG1 = [[val[1] for val in tag] for tag in tagsTrainG1]
tagCountG1 = Counter(list(itertools.chain.from_iterable(tagCountG1)))
tagCountG1

Counter({'chronic_disease': 4430,
         'cancer': 1454,
         'treatment': 5130,
         'allergy_name': 258})

Finding Distrubution of tag for each dataset

In [22]:
[x/sum(tagCountG1.values())*100 for x in tagCountG1.values()]

[39.300922640170334, 12.899219304471258, 45.51100070972321, 2.288857345635202]

In [23]:
tagCountG2 = [[val[1] for val in tag] for tag in tagsTrainG2]
tagCountG2 = Counter(list(itertools.chain.from_iterable(tagCountG2)))
tagCountG2

Counter({'allergy_name': 259,
         'cancer': 1471,
         'chronic_disease': 4206,
         'treatment': 4370})

In [24]:
[x/sum(tagCountG2.values())*100 for x in tagCountG2.values()]

[2.51309916553464, 14.273238889967011, 40.81117795458956, 42.40248398990879]

In [25]:

tagCountG3 = [[val[1] for val in tag] for tag in tagsTrainG3]
tagCountG3 = Counter(list(itertools.chain.from_iterable(tagCountG3)))
tagCountG3

Counter({'chronic_disease': 4023,
         'treatment': 4627,
         'cancer': 1327,
         'allergy_name': 325})

In [26]:
[x/sum(tagCountG3.values())*100 for x in tagCountG3.values()]

[39.050669772859635, 44.91360900795962, 12.880993981751116, 3.1547272374296256]

In [27]:

tagCountG4 = [[val[1] for val in tag] for tag in tagsTrainG4]
tagCountG4 = Counter(list(itertools.chain.from_iterable(tagCountG4)))
tagCountG4

Counter({'chronic_disease': 4430,
         'cancer': 1454,
         'treatment': 5130,
         'allergy_name': 258})

In [28]:
[x/sum(tagCountG4.values())*100 for x in tagCountG4.values()]

[39.300922640170334, 12.899219304471258, 45.51100070972321, 2.288857345635202]

In [29]:
# text = "human monster Hello I am a humanmonster"
# a=tokenizer(text, padding = "max_length", truncation = True, return_tensors = "pt",max_length=512)
# tokenizer.convert_ids_to_tokens(a['input_ids'][0])

While extending datasets, we ensure that disturbution of tags remaing same in the subset of 100 entries that we are appending. We also ensure, it has max tags to create a good training subset


In [30]:
#Keeping same distribution as G1/G2/G3
g1SubsetIndicesAllergy=np.argsort([-x.count("allergy_name") for x in [[val[1] for val in tag] for tag in tagsTrainG1]])[:3]
g2SubsetIndicesAllergy=np.argsort([-x.count("allergy_name") for x in [[val[1] for val in tag] for tag in tagsTrainG2]])[:3]
g3SubsetIndicesAllergy=np.argsort([-x.count("allergy_name") for x in [[val[1] for val in tag] for tag in tagsTrainG3]])[:3]


g1SubsetIndicesTreatment=np.argsort([-x.count("treatment") for x in [[val[1] for val in tag] for tag in tagsTrainG1]])[:45]
g2SubsetIndicesTreatment=np.argsort([-x.count("treatment") for x in [[val[1] for val in tag] for tag in tagsTrainG2]])[:42]
g3SubsetIndicesTreatment=np.argsort([-x.count("treatment") for x in [[val[1] for val in tag] for tag in tagsTrainG3]])[:45]


g1SubsetIndicesCancer=np.argsort([-x.count("cancer") for x in [[val[1] for val in tag] for tag in tagsTrainG1]])[:13]
g2SubsetIndicesCancer=np.argsort([-x.count("cancer") for x in [[val[1] for val in tag] for tag in tagsTrainG2]])[:14]
g3SubsetIndicesCancer=np.argsort([-x.count("cancer") for x in [[val[1] for val in tag] for tag in tagsTrainG3]])[:13]


g1SubsetIndicesChronicDisease=np.argsort([-x.count("chronic_disease") for x in [[val[1] for val in tag] for tag in tagsTrainG1]])[:39]
g2SubsetIndicesChronicDisease=np.argsort([-x.count("chronic_disease") for x in [[val[1] for val in tag] for tag in tagsTrainG2]])[:41]
g3SubsetIndicesChronicDisease=np.argsort([-x.count("chronic_disease") for x in [[val[1] for val in tag] for tag in tagsTrainG3]])[:39]

# print("Length of train G2 intially: "+str(len(tokensTrainG2)))
# tokensTrainG2.extend([tokensTrainG1[x] for x in g1SubsetIndicesAllergy])
# labelsTrainG2.extend([labelsTrainG1[x] for x in g1SubsetIndicesAllergy])

# tokensTrainG2.extend([tokensTrainG1[x] for x in g1SubsetIndicesTreatment])
# labelsTrainG2.extend([labelsTrainG1[x] for x in g1SubsetIndicesTreatment])

# tokensTrainG2.extend([tokensTrainG1[x] for x in g1SubsetIndicesCancer])
# labelsTrainG2.extend([labelsTrainG1[x] for x in g1SubsetIndicesCancer])

# tokensTrainG2.extend([tokensTrainG1[x] for x in g1SubsetIndicesChronicDisease])
# labelsTrainG2.extend([labelsTrainG1[x] for x in g1SubsetIndicesChronicDisease])
# print("Length of train G2 final: "+str(len(tokensTrainG2)))

# print("Length of train G3 intially: "+str(len(tokensTrainG3)))
# tokensTrainG3.extend([tokensTrainG1[x] for x in g1SubsetIndicesAllergy])
# tokensTrainG3.extend([tokensTrainG2[x] for x in g2SubsetIndicesAllergy])
# labelsTrainG3.extend([labelsTrainG1[x] for x in g1SubsetIndicesAllergy])
# labelsTrainG3.extend([labelsTrainG2[x] for x in g2SubsetIndicesAllergy])

# tokensTrainG3.extend([tokensTrainG1[x] for x in g1SubsetIndicesTreatment])
# tokensTrainG3.extend([tokensTrainG2[x] for x in g2SubsetIndicesTreatment])
# labelsTrainG3.extend([labelsTrainG1[x] for x in g1SubsetIndicesTreatment])
# labelsTrainG3.extend([labelsTrainG2[x] for x in g2SubsetIndicesTreatment])

# tokensTrainG3.extend([tokensTrainG1[x] for x in g1SubsetIndicesCancer])
# tokensTrainG3.extend([tokensTrainG2[x] for x in g2SubsetIndicesCancer])
# labelsTrainG3.extend([labelsTrainG1[x] for x in g1SubsetIndicesCancer])
# labelsTrainG3.extend([labelsTrainG2[x] for x in g2SubsetIndicesCancer])

# tokensTrainG3.extend([tokensTrainG1[x] for x in g1SubsetIndicesChronicDisease])
# tokensTrainG3.extend([tokensTrainG2[x] for x in g2SubsetIndicesChronicDisease])
# labelsTrainG3.extend([labelsTrainG1[x] for x in g1SubsetIndicesChronicDisease])
# labelsTrainG3.extend([labelsTrainG2[x] for x in g2SubsetIndicesChronicDisease])
# print("Length of train G3 final: "+str(len(tokensTrainG3)))


print("Length of train G4 intially: "+str(len(tokensTrainG4)))
tokensTrainG4.extend([tokensTrainG1[x] for x in g1SubsetIndicesAllergy])
tokensTrainG4.extend([tokensTrainG2[x] for x in g2SubsetIndicesAllergy])
tokensTrainG4.extend([tokensTrainG3[x] for x in g3SubsetIndicesAllergy])

labelsTrainG4.extend([labelsTrainG1[x] for x in g1SubsetIndicesAllergy])
labelsTrainG4.extend([labelsTrainG2[x] for x in g2SubsetIndicesAllergy])
labelsTrainG4.extend([labelsTrainG3[x] for x in g3SubsetIndicesAllergy])

tokensTrainG4.extend([tokensTrainG1[x] for x in g1SubsetIndicesTreatment])
tokensTrainG4.extend([tokensTrainG2[x] for x in g2SubsetIndicesTreatment])
tokensTrainG4.extend([tokensTrainG3[x] for x in g3SubsetIndicesTreatment])

labelsTrainG4.extend([labelsTrainG1[x] for x in g1SubsetIndicesTreatment])
labelsTrainG4.extend([labelsTrainG2[x] for x in g2SubsetIndicesTreatment])
labelsTrainG4.extend([labelsTrainG3[x] for x in g3SubsetIndicesTreatment])


tokensTrainG4.extend([tokensTrainG1[x] for x in g1SubsetIndicesCancer])
tokensTrainG4.extend([tokensTrainG2[x] for x in g2SubsetIndicesCancer])
tokensTrainG4.extend([tokensTrainG3[x] for x in g3SubsetIndicesCancer])


labelsTrainG4.extend([labelsTrainG1[x] for x in g1SubsetIndicesCancer])
labelsTrainG4.extend([labelsTrainG2[x] for x in g2SubsetIndicesCancer])
labelsTrainG4.extend([labelsTrainG3[x] for x in g3SubsetIndicesCancer])

tokensTrainG4.extend([tokensTrainG1[x] for x in g1SubsetIndicesChronicDisease])
tokensTrainG4.extend([tokensTrainG2[x] for x in g2SubsetIndicesChronicDisease])
tokensTrainG4.extend([tokensTrainG3[x] for x in g3SubsetIndicesChronicDisease])

labelsTrainG4.extend([labelsTrainG1[x] for x in g1SubsetIndicesChronicDisease])
labelsTrainG4.extend([labelsTrainG2[x] for x in g2SubsetIndicesChronicDisease])
labelsTrainG4.extend([labelsTrainG3[x] for x in g3SubsetIndicesChronicDisease])
print("Length of train G4 final: "+str(len(tokensTrainG4)))



Length of train G4 intially: 5884
Length of train G4 final: 6184


In [31]:
i=1211
wordLabelsTrainG1[i],wordsInTextTrainG1[i],tagsTrainG1[i]

({0: 'other',
  1: 'other',
  2: 'other',
  3: 'other',
  4: 'other',
  5: 'other',
  6: 'other',
  7: 'other',
  8: 'other',
  9: 'other',
  10: 'other',
  11: 'other',
  12: 'other',
  13: 'other',
  14: 'other',
  15: 'other',
  16: 'other',
  17: 'other',
  18: 'other',
  19: 'other',
  20: 'cancer'},
 ['for',
  'part',
  '2',
  'and',
  'part',
  '3',
  'of',
  'the',
  'study',
  ',',
  'participants',
  'must',
  'have',
  'histopathologically',
  'and',
  'clinically',
  'confirmed',
  'diagnosis',
  'of',
  'relapsed',
  'dlbcl'],
 array([['dlbcl', 'cancer']], dtype='<U6'))

Defining Dataset Class

In [2]:
class datasetNer(torch.utils.data.Dataset):
     def __init__(self, tokens,labels):
        self.tokens=tokens
        self.labels=labels
     def __len__(self):
        return len(self.tokens)
     def __getitem__(self, idx):
        token,label = self.tokens[idx],self.labels[idx]
        return token,label

Defining the model Class. We concatenate the last 2 layers of bio bert model and use it as input an ANN network of 4 layers

In [33]:

class nerModel(nn.Module):

  def __init__(self, numClasses):
    super(nerModel,self).__init__()
    self.num_labels = numClasses
    self.biobert = AutoModel.from_pretrained(modelName,output_hidden_states=True)
    self.fc1=nn.Linear(self.biobert.config.hidden_size*2,512)
    self.bn1 = nn.BatchNorm1d(512)
    self.fc2=nn.Linear(512,256)
    self.bn2 = nn.BatchNorm1d(512)
    self.fc3=nn.Linear(256,128)
    self.bn3 = nn.BatchNorm1d(512)
    self.dropout1=nn.Dropout(.1)
    self.dropout2=nn.Dropout(.1)
    self.gelu1 = nn.GELU()
    self.gelu3 = nn.GELU()
    self.gelu2 = nn.GELU()
    self.fc4=nn.Linear(128,numClasses)
  def forward(self, input_ids, attention_mask, labels = None):
        output=self.biobert(input_ids=input_ids, attention_mask=attention_mask,output_hidden_states=True)
        # print(output[0])
        # print(output.shape)
        logits = self.fc4(self.bn3(self.gelu3(self.fc3(self.dropout2(self.bn2(self.gelu2(self.fc2(
        self.dropout1(self.bn1(self.gelu1(self.fc1(torch.concat((output.hidden_states[-1],output.hidden_states[-2]),axis=2)))))))))))))
        if labels is not None:

            loss_fct = nn.CrossEntropyLoss(ignore_index=-100)
            # Only keep active parts of the loss
            if attention_mask is not None:
                active_loss = attention_mask.view(-1) == 1
                active_logits = logits.view(-1, self.num_labels)
                active_labels = torch.where(
                    active_loss, labels.view(-1), torch.tensor(loss_fct.ignore_index).type_as(labels)
                )
                loss = loss_fct(active_logits, active_labels)
            else:
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            return loss,logits
        return logits


In [34]:
def freeze_model(model,num_layers = 1):
    # print(len(list(model.parameters())))
    for id , params in enumerate(model.parameters()):
        # print(params.requires_grad)
        if id >= len(list(model.parameters())) - num_layers:
            params.requires_grad = True
        else:
            params.requires_grad = False
    return model

In [35]:
# lab=getLabelIds(tokensTrain[0].word_ids(),wordLabelsTrain[0])
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)
# model.train()
# o=model(tokensTrain[0]['input_ids'], attention_mask=tokensTrain[0]['attention_mask'],labels=torch.tensor(lab).to(device,dtype = torch.long).view(1, -1))

Defining a class to implement early stopping

In [36]:
class EarlyStopping(object):
    def __init__(self,patience,higher_is_better,tolerance,save_path,model):
        self.patience=patience
        self.higher_is_better = higher_is_better
        self.best_score= -100000 if self.higher_is_better else 100000
        self.tolerance=tolerance ### score should inc/dec by this much to by pass early stopping
        self.ctr=0
        self.save_path=save_path
        self.model=model

    def check(self,metric):
        if self.ctr<self.patience:

            if self.higher_is_better:
                if metric>=(self.best_score+self.tolerance):
                    self.best_score=metric
                    self.ctr =0
                    torch.save(self.model.state_dict(), self.save_path)
                else:
                     self.ctr += 1
            else:
                if metric<=(self.best_score-self.tolerance):
                    self.best_score=metric
                    self.ctr =0
                    torch.save(self.model.state_dict(), self.save_path)
                else:
                     self.ctr += 1
            return True

        else:
            return False

Funtion to get the desired metrics in a dictionary form

In [37]:

def getMetrics(predictions,labels):
    ignoreToken=-100
    predictions = predictions.flatten()
    labels = labels.flatten()

    predictions = predictions[labels != ignoreToken]
    labels = labels[labels != ignoreToken]

    predictions = predictions.to("cpu")
    labels = labels.to("cpu")

    acc = accuracy_score(labels,predictions)
    f1_scores = f1_score(labels, predictions, average = None)

    metrics ={}
    for class_id, f1 in enumerate(f1_scores):
        metrics["f1"+labelDict.get(class_id)]=f1

    f1_weighted_avg = f1_score(labels, predictions, average = "weighted")

    metrics["acc"] = round(acc ,3)
    metrics["f1_weighted_avg"]= round(f1_weighted_avg, 3)
    return metrics


In [38]:
# len(torch.tensor(getLabelIds(a[0].word_ids(),wordLabels[0])).to(dtype = torch.long))

In [39]:
# batchSize = 8
# testDSG1 = datasetNer(tokensTestG1, torch.tensor(labelsTestG1))
# testLoaderG1 = DataLoader(testDSG1, batch_size = batchSize, shuffle = False)

Merging test sets

In [40]:

tokensTestG4.extend(tokensTestG1)
tokensTestG4.extend(tokensTestG2)
tokensTestG4.extend(tokensTestG3)


labelsTestG4.extend(labelsTestG1)
labelsTestG4.extend(labelsTestG2)
labelsTestG4.extend(labelsTestG3)

Please input the path for your current model and the path to save the model for task T4

In [41]:
pathToCurrentModel = 'drive/MyDrive/nerDataset/nerModelG3.zip'
pathToSaveT4Model = 'drive/MyDrive/nerDataset/nerModelG4.zip'


In [42]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
trainDS = datasetNer(tokensTrainG4, torch.tensor(labelsTrainG4))
testDSG4 = datasetNer(tokensTestG4, torch.tensor(labelsTestG4))
valDS = datasetNer(tokensValG4, torch.tensor(labelsValG4))

batchSize = 8
max_lr=3e-4
early_stopping_rounds=2
maximize=True
num_epochs=20

trainLoader = DataLoader(trainDS, batch_size = batchSize, shuffle = False)
testLoaderG4 = DataLoader(testDSG4, batch_size = batchSize, shuffle = False)
valLoader = DataLoader(valDS, batch_size = batchSize, shuffle = False)


model = nerModel(len(uniqueLabels))
# model = freeze_model(model,100)
model = model.to(device)

model.load_state_dict(torch.load(pathToCurrentModel,map_location=device))

optimizer = AdamW(model.parameters(),lr=max_lr)
schedular=torch.optim.lr_scheduler.OneCycleLR(optimizer=optimizer,max_lr=max_lr,epochs=num_epochs,
                                        steps_per_epoch=len(trainDS))

es=EarlyStopping(patience=early_stopping_rounds,higher_is_better=1,tolerance=0.0001,
                save_path=pathToSaveT4Model,model=model)




cuda


In [43]:
model


nerModel(
  (biobert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affin

In [44]:
# tokensTrainG1[0]["input_ids"].to(device).shape
# tokensTrainG1[0]["attention_mask"].to(device).shape

In [45]:
# o=model(tokensTrainG1[0]["input_ids"].to(device),tokensTrainG1[0]["attention_mask"].to(device))

In [46]:
# tokensTrainG1[1]

In [47]:
# ceLoss(o.squeeze(),torch.tensor(labelsTrainG1[0]).squeeze())

In [48]:
# len(labelsTrainG1[0])

In [49]:
# def train(trainData,testData,epochs, optimizer):
#      for i in range(epochs):

#         model.train()
#         for xTrain, yTrain in tqdm(trainData):
#             optimizer.zero_grad()
#             # print(xTrain['input_ids'].squeeze(1).shape,xTrain['attention_mask'].squeeze(1).shape,yTrain.shape)

#             output= model(xTrain['input_ids'].squeeze(1).to(device), xTrain['attention_mask'].squeeze(1).to(device),
#                            yTrain.to(device))
#             loss=output.loss
#             logits=output.logits
#             print(loss.item())
#             loss.backward()
#             optimizer.step()

Validation loop to check early stopping

In [50]:
def validate_loop(valid_dl,model,model_dict):

    model.eval()
    with torch.no_grad():
        loss_=0
        pred_list=[]
        tar_list=[]
    preds=None
    labels=None
    total_loss_val =0
    j=0
    for X,Y in valid_dl:
        j+=1
        mask = X['attention_mask'].squeeze(1).to(device)
        input_id = X['input_ids'].squeeze(1).to(device)
        Y = Y.to(device)
        output = model(input_id, mask, Y)
        loss, logits = output
        if preds==None: preds = logits.argmax(dim= -1)
        else: preds = torch.cat((preds,logits.argmax(dim=-1)),0)
        if labels==None: labels = Y
        else: labels = torch.cat((labels,Y),0)
        total_loss_val += loss.item()

    valMetrics = getMetrics(preds,labels)
    # train_results = train_metrics.return_avg_metrics(len(trainData))
    print(f"Validation Loss: {total_loss_val / j}")
    print(valMetrics)
    model_dict['valid_params']['avg_loss'].append(total_loss_val /j)
    model_dict['valid_params']['metric'].append(valMetrics.get("f1cancer")+valMetrics.get("f1treatment")+valMetrics.get("f1chronic_disease")+valMetrics.get("f1allergy_name"))

    return model_dict

In [51]:
def init_state_dict():
    d={}
    # d['batch_num']=0
    d['valid_params'] = defaultdict(list)
    d['test_params'] = {}
    d['train_params'] =defaultdict(list)
    return d

In [52]:
def train(trainData,testData,valData,testG1,testG2,epochs, optimizer, schedular,early_stopping,ceLoss=None):
    model_dict=init_state_dict()
    for i in range(epochs):
        model.train() #train mode
        total_loss_train = 0
        j=0
        preds,labels=None,None
        for train_data, train_label in tqdm(trainData):
            j+=1
            train_label = train_label.to(device)
            '''
            squeeze in order to match the sizes. From [batch,1,seq_len] --> [batch,seq_len]
            '''
            mask = train_data['attention_mask'].squeeze(1).to(device)
            input_id = train_data['input_ids'].squeeze(1).to(device)

            optimizer.zero_grad()
            # model_dict['batch_num']+=1
            loss,logits = model(input_id, mask,train_label)
            # loss = ceLoss(logits.squeeze(),train_label.squeeze())
            if preds==None: preds = logits.argmax(dim= -1)
            else: preds = torch.cat((preds,logits.argmax(dim=-1)),0)
            if labels==None: labels = train_label
            else: labels = torch.cat((labels,train_label),0)
            # print(preds.shape,labels.shape)
            # #compute metrics
            # train_metrics.update(predictions, train_label)
            total_loss_train += loss.item()
            print(total_loss_train/j,i)
            #grad step
            loss.backward()
            optimizer.step()

        trainMetrics = getMetrics(preds,labels)
        # train_results = train_metrics.return_avg_metrics(len(trainData))
        print(f"Train Loss: {total_loss_train / j}")
        print(trainMetrics)
        model_dict['train_params']['avg_loss'].append(total_loss_train / j)
        model_dict['train_params']['metric'].append(trainMetrics.get("f1_weighted_avg"))
        model_dict['epoch_num']=i

        # if schedular is not None:
        #     schedular.step(metrics = val_metric)

        model_dict=validate_loop(valid_dl=valData,model=model,model_dict=model_dict)
        if(testG1 is not None):
            print("Evaluating on G1")
            eval(testG1,None,None,model,init_state_dict())
        if(testG2 is not None):
            print("Evaluating on G2")
            eval(testG2,None,None,model,init_state_dict())
        val_metric=model_dict['valid_params']['metric'][-1]
        if not early_stopping.check(val_metric):
            print("Early stopping at current epoch")
            break

    '''
    EVALUATION MODE
    '''
    model.eval()

    total_loss_test=0
    preds,labels=None,None
    j=0
    with torch.no_grad():
        for test_data, test_label in testData:
            j+=1

            test_label = test_label.to(device)

            mask = test_data['attention_mask'].squeeze(1).to(device)
            input_id = test_data['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask, test_label)
            loss, logits = output

            if preds==None: preds = logits.argmax(dim= -1)
            else: preds = torch.cat((preds,logits.argmax(dim=-1)),0)
            if labels==None: labels = test_label
            else: labels = torch.cat((labels,test_label),0)
            # dev_metrics.update(predictions, dev_label)
            total_loss_test += loss.item()
    testMetrics = getMetrics(preds,labels)
    print(f"Test Loss: {total_loss_test / j}" )
    print(testMetrics)
    model_dict['test_params']['avg_loss']=total_loss_test / j
    model_dict['test_params']['metric']=testMetrics
    return model_dict

In [ ]:
import warnings
warnings.filterwarnings("ignore")
model_dictG4=train(trainLoader,testLoaderG4,valLoader,None, None,num_epochs,optimizer,schedular,es)

Fucntion to evaluate on test data

In [54]:
def eval(testData1,testData2,testData3,model,model_dict):
    '''
    EVALUATION MODe
    '''
    model.eval()

    total_loss_test=0
    preds,labels=None,None
    j=0
    with torch.no_grad():
        for test_data, test_label in testData1:
            j+=1

            test_label = test_label.to(device)

            mask = test_data['attention_mask'].squeeze(1).to(device)
            input_id = test_data['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask, test_label)
            loss, logits = output

            if preds==None: preds = logits.argmax(dim= -1)
            else: preds = torch.cat((preds,logits.argmax(dim=-1)),0)
            if labels==None: labels = test_label
            else: labels = torch.cat((labels,test_label),0)
            # dev_metrics.update(predictions, dev_label)
            total_loss_test += loss.item()
    if(testData2 is not None):
        with torch.no_grad():
            for test_data, test_label in testData2:
                j+=1

                test_label = test_label.to(device)

                mask = test_data['attention_mask'].squeeze(1).to(device)
                input_id = test_data['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask, test_label)
                loss, logits = output

                if preds==None: preds = logits.argmax(dim= -1)
                else: preds = torch.cat((preds,logits.argmax(dim=-1)),0)
                if labels==None: labels = test_label
                else: labels = torch.cat((labels,test_label),0)
                # dev_metrics.update(predictions, dev_label)
                total_loss_test += loss.item()
    if(testData3 is not None):
        with torch.no_grad():
            for test_data, test_label in testData3:
                j+=1

                test_label = test_label.to(device)

                mask = test_data['attention_mask'].squeeze(1).to(device)
                input_id = test_data['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask, test_label)
                loss, logits = output

                if preds==None: preds = logits.argmax(dim= -1)
                else: preds = torch.cat((preds,logits.argmax(dim=-1)),0)
                if labels==None: labels = test_label
                else: labels = torch.cat((labels,test_label),0)
                # dev_metrics.update(predictions, dev_label)
                total_loss_test += loss.item()
    testMetrics = getMetrics(preds,labels)
    print(f"Test Loss: {total_loss_test / j}" )
    print(testMetrics)
    model_dict['test_params']['avg_loss']=total_loss_test / j
    model_dict['test_params']['metric']=testMetrics
    return model_dict

#Loading Saved Model

In [55]:
model.load_state_dict(torch.load(pathToSaveT4Model,map_location=device))


<All keys matched successfully>

#Final Results

In [56]:
model_dictG4 = eval(testLoaderG4,None,None,model,init_state_dict())

Test Loss: 0.2841113363067771
{'f1cancer': 0.8700920988851187, 'f1chronic_disease': 0.8733492097856679, 'f1treatment': 0.8615516135926481, 'f1allergy_name': 0.833780160857909, 'f1other': 0.9301340687209917, 'acc': 0.905, 'f1_weighted_avg': 0.906}
